In [1]:
# import python
import os
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import csv

In [2]:
# import R
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro

In [7]:
# import R packages

ro.r('library(spatstat)')
ro.r('library(sparr)')
ro.r('library(dplyr)')
ro.r('library(magrittr)')
ro.r('library(reshape2)')

'reshape2','magrittr','dplyr',...,'datasets','methods','base'


In [4]:
# read variables into R section
ro.r('h0 = 0.01')
ro.r('resolution = 256')

# hear I am reading 1/64 of the data I separated myself
ro.r('dataRAW <- read.csv("data/64split/rawdata_00_00_00.csv")')  

X,RA,DEC
...,...,...


In [6]:
# get single columns from data frame
ro.r('x <- dataRAW$RA')
ro.r('y <- dataRAW$DEC')

# put in the correct format
ro.r('myPointData <- ppp(x, y, xrange=range(x), yrange=range(y))')

# do KDE
ro.r('ddest <- bivariate.density(myPointData, h0=h0, adapt=TRUE, resolution=resolution)')

RParsingError: Parsing status not OK

In [ ]:
# output from KDE
ro.r('myDensEst <- ddest$z')

# define boundaries
ro.r('mins <- c(min(x), min(y))')
ro.r('maxs <- c(max(x), max(y))')

In [ ]:
# number of points to sample
ro.r('nPoints = 520000')

# normalize
ro.r('densEst <- ddest$kde2d/max(ddest$kde2d, na.rm=TRUE)')
     
# get dimension
ro.r('xdim <- dim(densEst)[1]')
ro.r('ydim <- dim(densEst)[2]')

ro.r('myNewXX <- vector(mode="numeric", length=nPoints)')
ro.r('myNewYY <- vector(mode="numeric", length=nPoints)')
ro.r('selPoints <- 1')


In [ ]:
data_raw = pd.read_csv('data/64split/rawdata_00_00_00.csv')

# Sample from density
for i in range(520000):
    ro.r('xx <- (runif(1) * xdim)')
    ro.r('yy <- (runif(1) * ydim)')
    ro.r('zz <- runif(1)')
    ro.r('myNewXX[selPoints] <- xx')
    ro.r('myNewYY[selPoints] <- yy')
    ro.r('selPoints <- selPoints + 1')

In [ ]:
# get new points into dataframe
ro.r('dataOUT = data.frame(x=myNewXX, y=myNewYY)')

In [ ]:
# write output to file
ro.r('write.csv(dataOUT, "from_KDE_00_00_00.csv")')

In [ ]:
# read output in python
data_drawn = pd.read_csv('test_00_00_00.csv')

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(1,2,1)
plt.title('original')
plt.scatter(data_raw['RA'], data_raw['DEC'], s=.01)

plt.subplot(1,2,2)
plt.title('drawn from KDE')
plt.scatter(data_drawn['x'],data_drawn['y'], s=.01)
plt.show()